[![Notebook Tutorial](__code/__all/notebook_tutorial.png)](https://neutronimaging.pages.ornl.gov/en/tutorial/notebooks/water_intake_profile_calculator/)

# Select your IPTS 

<img src='__docs/__all/shift_enter.gif' />

In [1]:
from __code import system
from __code.water_intake_profile_calculator import WaterIntakeProfileCalculator, WaterIntakeProfileSelector

from __code.ui_builder import UiBuilder
o_builder = UiBuilder(ui_name = 'ui_water_intake_profile.ui')
from __code.roi_selection_ui import Interface

system.System.select_working_dir()
from __code.__all import custom_style
custom_style.style()

pyuic5 /Users/j35/git/IPTS/python_notebooks/ui/ui_water_intake_profile.ui -o /Users/j35/git/IPTS/python_notebooks/__code/ui_water_intake_profile.py
** Using Debugging Mode! **


VBox(children=(HBox(children=(Label(value='IPTS-'), Text(value='', layout=Layout(width='10%')), Label(value='DOES NOT EXIST!', layout=Layout(width='20%'), _dom_classes=('result_label',)))), Label(value='OR'), HBox(children=(Label(value='Select Folder', layout=Layout(width='20%')), Select(layout=Layout(height='300px'), options=('BraggEdge', 'Brianne', 'DOE_letters', 'FITSimages', 'IPTS-12534', 'IPTS-13639_resonance_shawn', 'IPTS-14832', 'IPTS-15177', 'IPTS-16212', 'IPTS-16259', 'IPTS-16475', 'IPTS-16893', 'IPTS-17099', 'IPTS-17685', 'IPTS-17981', 'IPTS-18580', 'IPTS-18806', 'IPTS-18813', 'IPTS-19037', 'IPTS-19558', 'IPTS-19621', 'IPTS-19626', 'IPTS-20209-TOPAZ', 'IPTS_test_normalization', 'IPTS_testing_time_spectra', 'ImagesToRegisterForGian', 'Indu', 'Keita_code', 'Pour_ibeatles_video', 'ResonanceImaging', 'SENSITIVE', 'SarmaGUI', 'SphereCoordinates3D', 'TempData', 'gian', 'iBeatles_debugging', 'shared'), value='BraggEdge'))), Button(button_style='info', description='HELP', style=ButtonStyle())))

# Description 

This notebook will calculate the water intake profile vs time of a sample.

Here are the steps (**bold** for user input/manipulation)
 * **select the normalized images**
 * images sorted by time
 * **region of interest seleted**
 * profile of counts vs vertical-pixel calculated (**select integrated algorithm: mean/median/sum**)
 * water intake profile vs file index or vs time
 * **export profiles**

# Python Import 

In [2]:
%gui qt

In [3]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from ipywidgets.widgets import interact
import numpy as np
import os
import ipywe.fileselector
from ipywidgets import widgets
from IPython.core.display import display, HTML
import pytz
import datetime

from NeuNorm.normalization import Normalization

from __code.metadata_handler import MetadataHandler
from __code import file_handler


# Select Images to Process

In [4]:
o_water = WaterIntakeProfileCalculator(working_dir = system.System.get_working_dir())
o_water.select_data()

Button(button_style='info', description='HELP', style=ButtonStyle())

HTML(value='Please wait...')

VBox(children=(Label(value='Select Images ...', layout=Layout(width='250px')), HBox(children=(HBox(children=(Text(value='/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-15177', description='Location: ', layout=Layout(width='300px'), placeholder=''), Button(description='Jump', layout=Layout(margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px')),)), VBox(children=(SelectMultiple(description='Select', layout=Layout(display='flex', flex_flow='column', width='750px'), options=(' .', ' ..', ' .DS_Store                        |     Thu Feb 15 12:24:45 2018', ' Sample5_uptake_no bad images     |     Directory'), value=()), HBox(children=(Button(description='Enter directory', layout=Layout(margin='5px 40px'), style=ButtonStyle()), Button(description='Select', layout=Layout(margin='5px 40px'), style=ButtonStyle())))), layout=Layout(border='1px solid lightgrey', margin='5px', padding='10px'))))

HTML(value='Please wait...')

VBox(children=(Label(value='Select Images ...', layout=Layout(width='250px')), HBox(children=(HBox(children=(Text(value='/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-15177/Sample5_uptake_no bad images', description='Location: ', layout=Layout(width='300px'), placeholder=''), Button(description='Jump', layout=Layout(margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px')),)), VBox(children=(SelectMultiple(description='Select', layout=Layout(display='flex', flex_flow='column', width='750px'), options=(' .', ' ..', ' Sample5_1min_r_0.tif      |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_1.tif      |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_10.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_11.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_12.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_13.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_14.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_15.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_16.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_17.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_18.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_19.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_2.tif      |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_20.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_21.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_22.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_23.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_24.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_25.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_26.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_27.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_28.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_29.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_3.tif      |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_30.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_31.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_32.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_33.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_34.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_35.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_36.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_37.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_38.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_39.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_4.tif      |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_40.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_41.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_42.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_43.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_44.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_45.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_46.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_47.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_48.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_49.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_5.tif      |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_50.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_51.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_52.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_53.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_54.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_55.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_6.tif      |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_7.tif      |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_8.tif      |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_9.tif      |     Sun Mar

HBox(children=(Label(value='Retrieving Time Stamp', layout=Layout(width='20%')), IntProgress(value=0, layout=Layout(width='50%'), max=47)))

HBox(children=(Label(value='Loading sample', layout=Layout(width='20%')), IntProgress(value=0, max=47)))

# Select Profile Region 

In [5]:
o_gui = WaterIntakeProfileSelector(dict_data=o_water.dict_files)
o_gui.show()

/Users/j35/git/IPTS/python_notebooks/__code/water_intake_profile_calculator.py:568: RuntimeWarning: invalid value encountered in greater
  _result_range = np.where(image > range_max)


# DEBUGGING FROM HERE 

In [1]:
import numpy as np
from __code.water_intake_profile_calculator import MeanRangeCalculation

In [2]:
profile = [2.90287635,   2.8821102 ,   2.87690287,   2.89732621,
         2.88623043,   2.91406583,   2.95510404,   2.94335269,
         2.9679017 ,   3.04820217,   2.99743943,   3.01165945,
         2.99033492,   2.99397509,   2.9830557 ,   2.97041172,
         2.94597425,   2.92014064,   3.00387935,   2.98513203,
         2.96273962,   2.97548995,   2.95767174,   2.93331439,
         2.92838262,   2.96062786,   2.94428497,   2.89471726,
         2.95610289,   2.98263244,   2.89597947,   2.95887132,
         2.94526481,   2.91513829,   2.9107046 ,   2.90682367,
         2.98332211,   2.99965612,   3.00892162,   2.91350666,
         2.97087026,   2.97006539,   2.92745508,   2.91895019,
         2.9385388 ,   2.92364122,   2.9057143 ,   2.9323563 ,
         2.95392431,   3.02088575,   2.99304484,   2.92824206,
         3.01413435,   2.94689957,   2.92505252,   2.86290036,
         2.86386205,   2.92180767,   2.89750369,   2.89097811,
         2.8842245 ,   2.90745462,   2.93017139,   2.96633031,
         2.93905105,   2.94481158,   2.9414162 ,   2.91184672,
         2.93190604,   2.97820304,   2.96076195,   2.97923261,
         2.96735107,   2.90679909,   2.93514247,   2.91907445,
         2.87958034,   2.86164521,   2.91094531,   2.93996326,
         2.93099986,   2.86970711,   2.89143902,   2.9291985 ,
         2.94119958,   2.94881921,   2.92418828,   3.00228572,
         2.99929027,   2.99952063,   3.09081396,   3.12458999,
         3.1891421 ,   3.22690744,   3.32082677,   3.33318155,
         3.53689942,   3.69465733,   3.91474053,   4.29438958,
         5.13925313,   6.49579092,   8.4404762 ,  12.13579231,
        19.76926196,  32.89958793,  48.86539111,  61.01937041,
        70.94084078,  80.42760178,  88.00131834,  96.46191519,
       103.16454822, 110.79440224, 116.03331566, 122.75644809,
       127.37978226, 130.46286827, 132.37164587, 135.82572937,
       136.86968929, 137.15130579, 140.06299525, 141.09779632,
       143.31701672, 144.69096458,          np.NaN, 141.25618833,
       141.28930432, 142.09307724,          np.NaN, 140.85895902,
       142.30007625, 141.67516071, 141.74365395, 141.51511121,
       140.83288455,          np.NaN, 141.27556038, 141.12604433,
       141.47282237,          np.NaN, 140.4086768 , 141.29461122,
       143.09196746, 142.48738682, 142.36309993, 142.87076634,
       141.04375833, 139.43275625, 139.95112771, 140.4845432 ,
                np.NaN, 142.29233003, 141.89182144, 142.43202621,
       142.96775615, 141.10494447, 141.13946015, 141.16139859,
       141.52350342, 141.75551999, 142.34555137, 141.28567922,
       141.87249058, 142.38065147, 141.8494764 , 141.31270385,
       140.80789435,        np.NaN,          np.NaN]


In [4]:
_o_range = MeanRangeCalculation(data=profile)
for _pixel in np.arange(0, len(profile)-5):
    _o_range.calculate_left_right_mean(pixel=_pixel)

print(_o_range.left_mean)
print(_o_range.right_mean)
print(_o_range.calculate_delta_mean_square())

47.24889155378882
141.3233582
None
